## Wrap Up: Etherscan


In [1]:
url = "https://etherscan.io/"

Simple GET request without JS.

In [ ]:
import requests

html = requests.get(url)

html.content


With custom headers.

In [ ]:
hdr = {'User-Agent': 'Mozilla/5.0'}

html = requests.get(url, headers=hdr)

html.content


Improved GET request.

In [ ]:

import urllib.request

page = urllib.request.urlopen(url).read()

Improved GET request with custom headers.

In [5]:

from urllib.request import Request, urlopen

req = Request(url, headers = hdr)
page = urlopen(req)


Make soup.

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page)
soup

In [7]:
# Import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [8]:
# Load driver

# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument("--headless")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [9]:
driver.get(url)

Finding one specific element with XPath and class name.

_Note: the browser can export these selectors._

In [12]:
# CSS selector not available in Selenium.
# #mCSB_1_container > div:nth-child(1) > div.col-sm-4 > div > div.d-flex.flex-row.flex-sm-column.align-items-center.align-items-sm-start.gap-1.gap-sm-0 > a

my_xpath = '//*[@id="mCSB_1_container"]/div[1]/div[1]/div/div[2]/a'

elem = driver.find_element(By.XPATH, my_xpath)



In [ ]:
elem.text

In [ ]:
elem = driver.find_element(By.CLASS_NAME, 'hash-tag')

elem.text


In [ ]:
elements = driver.find_elements(By.CLASS_NAME, 'hash-tag')

In [ ]:
type(elements), len(elements)

#### All transactions

Let's now get all transactions, not just what is on the home page.


In [ ]:
url = 'https://etherscan.io/txs'

By visual inspection in DevTools, each row contains 3 elements:
- transaction hash
- from
- to

In [ ]:
driver.get(url)
elements = driver.find_elements(By.CLASS_NAME, 'hash-tag')

In [ ]:
# Let's prepare for it.

transactions = []
toAdx = ''
fromAdx = ''

## Go through all elements containing an address or a transaction hash.
for i in range(len(elements)):
    adx = elements[i-1].text
    if i % 2 and not i % 3 == 0:
        fromAdx = adx
    if i % 3 == 0:
        toAdx = adx
        transactions.append((fromAdx, toAdx))

print(transactions)

print(len(transactions))

Let's build some simple statistics, counting the number of times an address receive/send a transaction. 


In [ ]:
from collections import defaultdict
sentCounts = defaultdict(int)
receivedCounts = defaultdict(int)

for i in range(len(elements)):
    adx = elements[i-1].text
    if i % 2 and not i % 3 == 0:
        sentCounts[adx] += 1
    if i % 3 == 0:
        receivedCounts[adx] += 1


print('Sent Counts')
print(sentCounts)
print('Received Counts')
print(receivedCounts)

In [ ]:
# Support functions.

from datetime import datetime

def print_time():
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("time: ", dt_string)

def update_to_from(elements):
    for i in range(len(elements)):
        adx = elements[i-1].text
        if i % 2 and not i % 3 == 0:
            sentCounts[adx] += 1
        if i % 3 == 0:
            receivedCounts[adx] += 1


In [ ]:
# Fetch multiple pages of transactions.

url = 'https://etherscan.io/txs?ps=100&p='

# Reset counts.
sentCounts = defaultdict(int)
receivedCounts = defaultdict(int)

for i in range(1,4):
    my_url = url + str(i)
    driver.get(my_url)
    elements = driver.find_elements(By.CLASS_NAME, 'hash-tag')
    update_to_from(elements)
    print_time()



In [ ]:
def printSorted(dic):
    sorted_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    for i in sorted_dic:
        print(i[0], i[1])


In [ ]:
printSorted(sentCounts)


In [ ]:
printSorted(receivedCounts)

Let's gather more pages of transactions.

We can also try out different methods.

In [ ]:
def getTxs(n=3, reset=True, showCounts=True, method="requests"):
    # Access global variables from within a function's body.
    global sentCounts
    global receivedCounts

    url = 'https://etherscan.io/txs?ps=100&p='

    # Reset counts.
    if reset:
        print('Counts reseted')
        sentCounts = defaultdict(int)
        receivedCounts = defaultdict(int)

    for i in range(1,n+1):
        my_url = url + str(i)

        if method == "requests":
            html = requests.get(my_url, headers=hdr)
            soup = BeautifulSoup(html.content, 'html.parser')
            elements = soup.select(".hash-tag")
        elif method == "selenium":
            driver.get(my_url)
            elements = driver.find_elements(By.CLASS_NAME, 'hash-tag')
        elif method == "urllib":
            req = Request(url, headers = hdr)
            page = urlopen(req)
            soup = BeautifulSoup(page, 'html.parser')
            elements = soup.select(".hash-tag")

        update_to_from(elements) 
        print_time()
    
    if showCounts:
        printSorted(sentCounts)
        printSorted(receivedCounts)
 

In [ ]:
getTxs(1, method = "selenium", showCounts = False)

In [ ]:
getTxs(1, method = "requests", showCounts = False)

In [ ]:
getTxs(1, method = "urllib", showCounts = False)

In [ ]:
printSorted(sentCounts)

In [ ]:
printSorted(receivedCounts)

In [ ]:
import pandas as pd

In [ ]:
# Sent.
items = sentCounts.items()
df_sent = pd.DataFrame.from_dict(items)
df_sent.columns = ["address", "sent"]
df_sent.sort_values(by="sent", ascending=False)

In [ ]:
# Sent.
items = receivedCounts.items()
df_received = pd.DataFrame.from_dict(items)
df_received.columns = ["address", "received"]
df_received.sort_values(by="received", ascending=False)

In [ ]:
df_txs = pd.merge(df_sent, df_received, on = "address", how = "outer")
df_txs = df_txs.fillna(0)
df_txs.sort_values(by="received", ascending=False)

In [ ]:
df_txs.plot.scatter(x='sent',
                    y='received',
                    c='DarkBlue')

In [ ]:
import random

def jitter(x):
    return x + random.uniform(0, .5) -.25

df_txs['sent_jit'] = df_txs['sent'].apply(lambda x: jitter(x))
df_txs['received_jit'] = df_txs['received'].apply(lambda x: jitter(x))

In [ ]:
df_txs.plot.scatter(x='sent_jit',
                    y='received_jit',
                    c='DarkBlue')